In [1]:
import sys
sys.path.append("../")
import data.directrepository as dr
import data.flight_repository as fr
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
import importlib as im


#im.reload(data.directrepository)
#im.reload(data.flight_repository)

In [2]:
%time directs = dr.DirectsRepository().get_directs()
directs[:2]

Wall time: 91.5 ms


,Id,FlyFrom,FlyTo
0,1,FLR,AMS
1,2,FLR,ANR


In [3]:
#%time flights = fr.FlightsRepository().get_flights()
#flights[:2]

In [4]:
#table = pa.Table.from_pandas(flights)
#pq.write_to_dataset(table, root_path='allflights.parquet' )

In [5]:
%time flights = pq.read_table("allflights.parquet").to_pandas()
flights[:2]


Wall time: 364 ms


,Id,FlyFrom,FlyTo,Price,Airline,ArrivalTimeUTC,DepartTimeUTC,FlightNum
0,1,JFK,LHR,362.0,VS,2019-10-17 05:50:00,2019-10-16 23:00:00,4030
1,2,JFK,LHR,362.0,VS,2019-10-16 05:50:00,2019-10-15 23:00:00,4030


In [6]:
# get directs from target
%time dir_tar = directs[directs['FlyFrom'] == "FLR" ]
dir_tar[:2]

Wall time: 2 ms


,Id,FlyFrom,FlyTo
0,1,FLR,AMS
1,2,FLR,ANR


In [7]:
import queue
from enum import Enum
sys.path.append("../../")
import services.core.recoservice_df as reco
#im.reload(services.core.recoservice_df)
from model.flight import Flight
from model.direct import Direct
import datetime

In [13]:
f1 = Flight("JFK", "LHR", 1000, "DU", "", datetime.datetime(2019, 5, 10, 11, 0, 0),
            datetime.datetime(2019, 5, 10, 6, 0, 0), "123")
f2 = Flight("JFK", "MIA", 50, "DU", "", datetime.datetime(2019, 5, 5, 11, 0, 0),
            datetime.datetime(2019, 5, 5, 6, 0, 0), "456")
f3 = Flight("MIA", "LHR", 100, "DU", "", datetime.datetime(2019, 5, 8, 11, 0, 0),
            datetime.datetime(2019, 5, 8, 6, 0, 0), "456")
f4 = Flight("LHR", "EWR", 450, "DU", "", datetime.datetime(2019, 5, 18, 11, 0, 0),
            datetime.datetime(2019, 5, 18, 6, 0, 0), "123")
f5 = Flight("LHR", "ATL", 200, "DU", "", datetime.datetime(2019, 5, 18, 11, 0, 0),
            datetime.datetime(2019, 5, 18, 6, 0, 0), "123")
f6 = Flight("ATL", "JFK", 300, "DU", "", datetime.datetime(2019, 5, 20, 11, 0, 0),
            datetime.datetime(2019, 5, 20, 6, 0, 0), "123")
f7 = Flight("EWR", "LHR", 1200, "DU", "", datetime.datetime(2019, 5, 16, 11, 0, 0),
            datetime.datetime(2019, 5, 16, 6, 0, 0), "123")
f8 = Flight("JFK", "LHR", 1200, "DU", "", datetime.datetime(2019, 5, 16, 11, 0, 0), #dup of f1 but higher price
            datetime.datetime(2019, 5, 16, 6, 0, 0), "123")
f9 = Flight("JFK", "LHR", 5, "DU", "", datetime.datetime(2019, 5, 10, 11, 0, 0), #dup of f1 but on same day
            datetime.datetime(2019, 5, 10, 6, 0, 0), "123")
fs = [f1, f2, f3, f4, f5, f6, f7, f8,f9]
flights = pd.DataFrame.from_records([f.to_dict() for f in fs])

sources = ["JFK","EWR"]
targets = ["LHR"]
exclusions = []
q = reco.RecommendationService().get_initial_queue(sources)
for elem in list(q.queue):
    print(str(elem))


JFK at_source 
EWR at_source 


In [34]:
current = q.get()
print("Current: " + current.current_loc + " " + current.state.name)
for elem in list(q.queue):
    print(str(elem))

Current: LHR at_target
LHR at_target  | JFK -> MIA | MIA -> LHR


In [17]:
fs = flights
int_destinations = fs[fs["FlyTo"].isin(targets)]["FlyFrom"].unique()
f1 = fs[(fs["FlyFrom"] == current.current_loc) & (~fs["FlyTo"].isin(exclusions)) & ((fs["FlyTo"].isin(int_destinations)) | (fs["FlyTo"].isin(targets)))]
f1 = f1.assign(rn = f1.sort_values(['Price']).groupby(['FlyTo']).cumcount()+1).query('rn < 2')
for index, first_stop in f1.iterrows():
    flight_state = reco.FlightState.at_intermediate_1
    if first_stop["FlyTo"] in targets:
        flight_state = reco.FlightState.at_target

    new_path = current.current_path.copy()
    new_path.append(first_stop)

    q.put(reco.FlightItem(first_stop["FlyTo"], new_path, flight_state))
    
for elem in list(q.queue):
    print(str(elem))


MIA at_intermediate_1  | JFK -> MIA
LHR at_target  | JFK -> LHR
LHR at_target  | EWR -> LHR


In [33]:
#at target
fs = flights
prev_arrival_time = current.current_path[len(current.current_path)-1]["ArrivalTimeUTC"]
int_destinations = fs[fs["FlyFrom"].isin(targets)]["FlyTo"].unique()
f1 = fs[(fs["FlyFrom"] == current.current_loc) & 
        (fs["DepartTimeUTC"] > prev_arrival_time + datetime.timedelta(days=2)) & 
        (fs["DepartTimeUTC"] < prev_arrival_time + datetime.timedelta(days=8)) &
        (~fs["FlyTo"].isin(exclusions)) & 
        ((fs["FlyTo"].isin(int_destinations)) | (fs["FlyTo"].isin(sources)))]
f1 = f1.assign(rn = f1.sort_values(['Price']).groupby(['FlyTo']).cumcount()+1).query('rn < 2')




,Airline,ArrivalTimeUTC,DepartTimeUTC,Duration,FlightNum,FlyFrom,FlyTo,Price,rn
3,DU,2019-05-18 11:00:00,2019-05-18 06:00:00,,123,LHR,EWR,450,1
4,DU,2019-05-18 11:00:00,2019-05-18 06:00:00,,123,LHR,ATL,200,1


In [73]:
current.current_path[]


[Airline                            DU
 ArrivalTimeUTC    2019-05-05 11:00:00
 DepartTimeUTC     2019-05-05 06:00:00
 Duration                             
 FlightNum                         456
 FlyFrom                           JFK
 FlyTo                             MIA
 Price                              50
 rn                                  1
 Name: 1, dtype: object]

In [19]:
#at int1

prev_arrival_time = current.current_path[len(current.current_path)-1]["ArrivalTimeUTC"]
fs = flights
f1 = fs[(fs["FlyFrom"] == current.current_loc) & (fs["FlyTo"].isin(targets)) & 
        (fs["DepartTimeUTC"] > prev_arrival_time + datetime.timedelta(days=2)) & 
        (fs["DepartTimeUTC"] < prev_arrival_time + datetime.timedelta(days=3))]
f1 = f1.assign(rn = f1.sort_values(['Price']).groupby(['FlyTo']).cumcount()+1).query('rn < 2')

for index, first_stop in f1.iterrows():
    new_path = current.current_path.copy()
    new_path.append(first_stop)

    q.put(reco.FlightItem(first_stop["FlyTo"], new_path, reco.FlightState.at_target))
    
for elem in list(q.queue):
    print(str(elem))

AttributeError: 'FlightItem' object has no attribute 'print'